In [9]:
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')
import itertools
import anndata
import pandas as pd
# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *


In [10]:
seed = 1
print(f"Using seed: {seed}")

outfile = f"./testing_results_pseudo/seed_{seed}/CRE_gene_cell_population.tsv"

Using seed: 1


In [11]:
results = pd.read_csv(outfile, sep ='\t',index_col = 0)


In [12]:
split_columns = results['CRE'].str.split(':|-', expand=True)
split_columns.columns = ['chr', 'start', 'end']  # Rename the new columns
results = pd.concat([results, split_columns], axis=1)
grouped = results.groupby("Gene")
results = results[['chr', 'start', 'end']]
print(results)


         chr     start       end
0       chr1    778219    779393
1       chr1    778219    779393
2       chr1    826772    827922
3       chr1    826772    827922
4       chr1    826772    827922
...      ...       ...       ...
188301  chrY  12904476  12905923
188302  chrY  13282692  13283155
188303  chrY  14524430  14524989
188304  chrY  19744506  19745162
188305  chrY  20575346  20576097

[188306 rows x 3 columns]


In [13]:
import os
outDir =f'./testing_results_pseudo/seed_{seed}/TF_motif/bedfile/'
if not os.path.exists(outDir):
    print(outDir)
    os.makedirs(outDir)

In [14]:
top_percentile = 0.2

for category, group in grouped:

    if "/" in category:
        continue
    threshold_value = group['rss_ratio'].quantile(1 - top_percentile)

    top_rows = group[group['rss_ratio'] >= threshold_value]
    top_rows = top_rows[['chr', 'start', 'end']]
    top_rows.to_csv(outDir+category+".bed",sep="\t",index=False)


KeyboardInterrupt



In [8]:
import pyranges as pr
import os
path_to_region_sets = outDir
print(path_to_region_sets)
region_sets_files = os.listdir(path_to_region_sets)
region_sets = {}
i=0
for x in region_sets_files:

    temp_doc = pr.read_bed(os.path.join(path_to_region_sets, x)) 
    if len(temp_doc) <10:
        continue
    else:
        i+=1
#         print(len(temp_doc),i)
        region_sets[x.replace('.bed', '')] = temp_doc


./results/TF_motif/bedfile/


In [9]:
db = './data/hg38_screen_v10_clust.regions_vs_motifs.rankings.feather'

In [10]:
def identify_all_TF_gene_link_candidates(rna_TF_dictionary):

    # Flattening the dictionary
    flat_data = [(v, k) for k, lst in rna_TF_dictionary.items() for v in lst ]

    # Creating DataFrame
    df = pd.DataFrame(flat_data, columns=['TF', 'Gene'])
    return df

In [11]:
def read_dictionary_in_chunks(dictionary, chunk_size, db, specie='homo_sapiens', outDir=outDir):
    keys = list(dictionary.keys())
    print(f"Total keys: {len(keys)}")
    
    os.makedirs(outDir, exist_ok=True)  # Ensure output directory exists

    for i in range(0, len(keys), chunk_size):
        print(f"{i}")
        chunk = {key: dictionary[key] for key in keys[i:i + chunk_size]}

        try:
            print("Initializing database...")
            ctx_db = cisTargetDatabase(db, chunk)

            print("Running cisTarget...")
            cistarget_dict = run_cistarget(
                ctx_db=ctx_db,
                region_sets=chunk,
                specie=specie,
                annotation_version='v10nr_clust',
                auc_threshold=0.005,
                nes_threshold=3.0,
                rank_threshold=0.05,
                annotation=['Direct_annot', 'Orthology_annot'],
                n_cpu=18,
                _temp_dir='/users/tpham43/'
            )
            
            print("Processing cisTarget results...")
            filtered_data = {
                key: [TF.split('_')[0] for TF in values.cistromes['Region_set'].keys()]
                for key, values in cistarget_dict.items()
            }

            output_file = os.path.join(outDir, f"TF_gene_OT_{i}.csv")
            print(f"Writing results to {output_file}")
            identify_all_TF_gene_link_candidates(filtered_data).to_csv(output_file)

        except Exception as e:
            print(f"Error processing chunk starting at index {i}: {e}")
            continue

In [8]:
import ray
ray.shutdown()
read_dictionary_in_chunks(region_sets,200,db =db)

NameError: name 'read_dictionary_in_chunks' is not defined

In [20]:
# Specify the folder path
folder_path = outDir

# Get a list of all files in the folder
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Filter the list for CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Read all CSV files into a list of DataFrames
dataframes = [pd.read_csv(f,index_col=0) for f in csv_files]

# Combine all DataFrames into a single DataFrame
candidate_df = pd.concat(dataframes, ignore_index=True)

candidate_df = candidate_df.drop_duplicates()
rna_anndata = sc.read_h5ad("./data/rna_data_filtered_pca_1control_5cells_real.h5ad")

# rna_anndata = sc.read_h5ad('./data/rna_downsampled_snare.h5ad')

# Check if raw data exists
if rna_anndata.raw is not None:
    # Access the raw data
    raw_data = rna_anndata.raw.X
    raw_obs = rna_anndata.obs.copy()  # Cell metadata (unchanged)
    raw_var = rna_anndata.raw.var.copy()  # Gene metadata (original raw genes)

    # Create a new AnnData object using the raw data
    rna_anndata = sc.AnnData(X=raw_data, obs=raw_obs, var=raw_var)

    # Print the new raw AnnData object to verify
    print(rna_anndata)

candidate_df = candidate_df[candidate_df['Gene'].isin(rna_anndata.var.index.values)]

candidate_df.rename(columns={'TF': 'Source', 'Gene': 'Target'}, inplace=True)

candidate_df.to_csv(f"./testing_results_pseudo/seed_{seed}/TF_motif/putative_TF_gene.tsv",sep = '\t')

print("Done")


Done
